In [1]:
from collections import defaultdict
import random

In [2]:
def DeBruijnFromkMers(Patterns):
    dB = defaultdict(list)
    for i in range(0,len(GappedPatterns),2):
        firstPattern = GappedPatterns[i]
        secondPattern = GappedPatterns[i+1]
        first_prefix = firstPattern[:-1]
        first_suffix = firstPattern[1:]
        second_prefix = secondPattern[:-1]
        second_suffix = secondPattern[1:]
        dB[(first_prefix,second_prefix)].append((first_suffix,second_suffix))
    return dB

In [3]:
def findGlobalStartEndNode(Graph):
    in_out_edges_diff = {}
    for key,values in Graph.items():
        if key in in_out_edges_diff.keys():
            in_out_edges_diff[key] += -len(values)
        else:
            in_out_edges_diff[key] = -len(values)
        for val in values:
            if val in in_out_edges_diff.keys():
                in_out_edges_diff[val] += 1
            else:
                in_out_edges_diff[val] = 1
    global_end_node = [k for k, v in in_out_edges_diff.items() if v == 1][0]
    global_start_node  = [k for k, v in in_out_edges_diff.items() if v == -1][0]
    
    Graph[global_end_node] = {global_start_node}
    return global_start_node,global_end_node

In [4]:
def randomWalk(Graph,Cycle):
    start_node = Cycle[0]
    node = start_node
    while Graph[node]:
        ####Note! it removes an arbitrary (but not random) set value.
        ####BUT i think it still works!
        new_node = Graph[node].pop()
        if(not Graph[node]): ###Empty value sets
            del Graph[node]  ###Remove the previous node
        Cycle.append(new_node)
        node = new_node
    del Graph[node]   #####Remove the start node without unsed nodes
    return Cycle

In [5]:
def EulerianPath(Graph):
    global_start_node,global_end_node = findGlobalStartEndNode(Graph)
    Cycle = randomWalk(Graph,[global_start_node])
    return Cycle

In [6]:
def  PathToGenome(kmers):
    Text = kmers[0]
    n = len(kmers)
    for i in range(1,n):
        Text += kmers[i][-1]
    return Text

In [7]:
def StringSpelledByGappedPatterns(GappedPatterns, k, d):
    #FirstPatterns ← the sequence of initial k-mers from GappedPatterns
    FirstPatterns = [k[0] for k in GappedPatterns[:-1]]
    #SecondPatterns ← the sequence of terminal k-mers from GappedPatterns
    SecondPatterns = [k[1] for k in GappedPatterns[:-1]]
    #PrefixString ← StringSpelledByGappedPatterns(FirstPatterns, k)
    PrefixString = PathToGenome(FirstPatterns)
    #SuffixString ← StringSpelledByGappedPatterns(SecondPatterns, k)
    SuffixString = PathToGenome(SecondPatterns)
    
    for i in range(k+d,len(PrefixString)):
        #if the i-th symbol in PrefixString does not equal the (i - k - d)-th symbol in SuffixString
        if (PrefixString[i] != SuffixString[i-k-d]):
            return "there is no string spelled by the gapped patterns"
        return PrefixString+SuffixString[-(k+d):]

In [8]:
def StringReconstruction(Patterns,k,d):
        dB =  DeBruijnFromkMers(Patterns)
        path = EulerianPath(dB)
        Text = StringSpelledByGappedPatterns(path,k-1,d+1)  ###Reduce to k-1 mer, distance became d+1
        return Text

In [9]:
GappedPatterns = ['GAGA','TTGA',
'TCGT','GATG',
'CGTG','ATGT',
'TGGT','TGAG',
'GTGA','TGTT',
'GTGG','GTGA',
'TGAG','GTTG',
'GGTC','GAGA',
'GTCG','AGAT']


In [10]:
Text = StringReconstruction(GappedPatterns,4,2)
Text

'GTGGTCGTGAGATGTTGA'

In [11]:
GappedPatterns = []
k = 0
d = 0
file = open('dataset_369274_16.txt', 'r') 
for i, line in enumerate(file):
    line=line.rstrip('\n')
    if (i == 0):
        k,d = line.split(' ')
        k = int(k)
        d = int(d)
    else:
        first,second = line.split('|')
        GappedPatterns.append(first)
        GappedPatterns.append(second)

In [12]:
Text = StringReconstruction(GappedPatterns,k,d)
Text

'AGGAGGCGAACAGATGCGCCTGCAAGTGTAGGCATCTCAGGGAATTTTGACTCATTAGCGAGTCTGTTTCCGAAGTGGCATTATTTACATACCATCGTCCTCCATAGTAGTCAGGAGGTTTTCGCCTGCAGCAGAGAACGGCAGCCTTTAACATCGACTGTACGGTCCCTTTACCATCGCAGCTGACCTTGTTCGCGACACTGACAAAAAACACAGACTCAGTGACAACGCGCTGAGGTTGCGGCAAAAAATTCTCGCGATCCCCGGTCCAGCAGCACCAGAGCTAAAAGTAGTCAGGAGGTTTTCGCCTGCAGCAGAGAACGGCAGCCTTTAACATCTTCGTGCCCAGTATTTGCTTATAGATGGGAACAACCTATTCACAGCGTTTGCCATCCTGTTTATCTCTGACGGCATTTAGCACAGTAGGTGGCAAGGATCGCGCCCTATGCACGAGTAGTCAGGAGGTTTTCGCCTGCAGCAGAGAACGGCAGCCTTTAACATCGTTGTATCGTACAGTAGTCAGGAGGTTTTCGCCTGCAGCAGAGAACGGCAGCCTTTAACATCATTAGTCAAATTAACCCACATACAGCAAGTAGTCAGGAGGTTTTCGCCTGCAGCAGAGAACGGCAGCCTTTAACATCAGAAAACCGAATACCTGGATACTACTTGCTGGTAAGCAAACCATATGCGCTCCTTTCCTCTGCCCACGTTGTTAAAGAAAGACCGAACGCCTCCCAGGTCCGCCAGTTGTTAAGCGGCGGAATGGCTATAATAAGTGACTAGTTGAACGTTGTGGTGCAGTCACCCGAGGTGATGCTGTGCTACACTTTGGTAACGGAGCACCGATGCCTCTCGCCGCCTGGGTATCAGCGGTGGTCTCACAGCCCTGTCACGACAAGGCGATCACCGACTTACCGGGATGCAGGACCTCGTACGCGACTCAAATAAGTAGAGTAAATAACTATCGTGTATAACGGAGTACGCTTCTTAGTTCAGCAA